# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Phan Nguyễn Phước Nguyên
- MSSV: 20127577
- Lớp: 20CLC11

# Import

In [639]:
import pandas as pd
import numpy as np

# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [640]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)


# Cài đặt hàm

In [641]:
# Cài đặt các hàm cần thiết ở đây
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y
        return self   
    def get_params(self):
        return self.w
    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)   

# Yêu cầu 1a: Sử dụng toàn bộ 10 đặc trưng đề bài cung cấp (2 điểm) 

In [642]:
# Phần code cho yêu cầu 1a
lr = OLSLinearRegression().fit(X_train, y_train)
y_predict = lr.predict(X_test)

In [643]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
rms = rmse(y_predict,y_test)

Công thức hồi quy

$$\text{Life expectancy} =  𝛽0 + 𝛽1x1 + ⋯ + 𝛽10𝑥10 + 𝜀$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng, tìm mô hình cho kết quả tốt nhất (2 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [644]:
# Phần code cho yêu cầu 1b
from sklearn.model_selection import KFold

# train/test tren 5fold va tra ve gia tri tb cua 5 lan 
def rmse_mean_cal(model, X, X_test, y, kfold):
    # tinh rmse tung fold
    rms = 0
    for train_ids, val_ids in kfold.split(X, y):
        model.fit(X[train_ids], y[train_ids])
        y_predict = model.predict(X_test)
        rms = rms + rmse(y_predict,y_test)
    #lay trung binh
    return rms/5    

# concat phan train va test lai 
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

# chia thanh 5 fold
kfold = KFold(n_splits=5,random_state=1, shuffle=True)

# In ra các kết quả cross-validation như yêu cầu
best_idx = 0
temp = 100
best_rms = 0
for i in range(0,10):
    if(i==0):
        print('{0: <5}'.format("STT"), '{0: <35}'.format("Mô hình với 1 đặc trưng"), "RMSE")
    col = [i]
    X1_train = train.iloc[:,col].to_numpy()
    X1_test = test.iloc[:,col].to_numpy()
    y1_train = y_train.to_numpy()
    rms = rmse_mean_cal(OLSLinearRegression(), X1_train, X1_test, y1_train, kfold)
    if(rms < temp):
        best_idx = i
        temp = rms
        best_rms = rms
    print('{0: <5}'.format(i+1), '{0: <35}'.format(train.columns[i]), rms)
    

# Tìm ra đặc trưng tốt nhất
print("Đặc trưng tốt nhất:", train.columns[best_idx],":", best_rms)


STT   Mô hình với 1 đặc trưng             RMSE
1     Adult Mortality                     45.62493038094085
2     BMI                                 26.21729921152169
3     Polio                               17.98546983493596
4     Diphtheria                          15.553270331802272
5     HIV/AIDS                            69.02769971532624
6     GDP                                 59.711600292023334
7     Thinness age 10-19                  53.44080927591617
8     Thinness age 5-9                    52.90478883918289
9     Income composition of resources     9.463088664795228
10    Schooling                           10.26194576513432
Đặc trưng tốt nhất: Income composition of resources : 9.463088664795228


In [645]:
# Huấn luyện lại mô hình best_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
X1_train = train.iloc[:,[best_idx]].to_numpy()
X1_test = test.iloc[:,[best_idx]].to_numpy()

lr.fit(X1_train, y_train) 
y_predict = lr.predict(X1_test)

In [646]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_feature_model
rms = rmse(y_predict, y_test)
rms

9.462641625274049

Công thức hồi quy

$$\text{Life expectancy} = 𝛽0 + 𝛽9x9 + 𝜀$$

# Yêu cầu 1c: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

In [647]:
# Phần code cho yêu cầu 1c
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)

# chuan hoa (standardize)
X2_train = X_train
X2_test = X_test
scale= StandardScaler()
X2_train = scale.fit_transform(X2_train) 
X2_test = scale.fit_transform(X2_test) 

# convert back to df
X2_train = pd.DataFrame(X2_train, columns = X_train.columns)
X2_test = pd.DataFrame(X2_test, columns = X_train.columns)
X_train['NewCol10'] = (X2_train['Income composition of resources'] + X2_train['Schooling']) 
X_test['NewCol10'] = (X2_test['Income composition of resources'] + X2_test['Schooling']) 

X_train['NewCol11'] = (0.5 * X2_train['GDP'] + X2_train['Schooling']) 
X_test['NewCol11'] = (0.5 * X2_test['GDP'] + X2_test['Schooling']) 

#lay ngau nhien 3 feature 0,5,6
model1 = [0, 5, 6] 
#tao 1 feature moi (NewCol10) bang tong 2 feature Income composition of resources va Schooling (sau khi chuan hoa)
model2 = [10]    
#tao 1 feature moi (NewCol11) bang tong 2 feature 0.5GDP va Schooling
#sau do ket hop voi cac feature tu model 1
model3 = [0, 5, 6, 8, 11]       
models = [model1, model2, model3]


# In ra các kết quả cross-validation như yêu cầu
for i in range(0, len(models)):
    if(i==0):
        print('{0: <5}'.format("STT"), '{0: <35}'.format("Mô hình"), "RMSE")
    col = models[i]
    X2_train = X_train.iloc[:,col].to_numpy()
    X2_test = X_test.iloc[:,col].to_numpy()
    y1_train = y_train.to_numpy()
    rms = rmse_mean_cal(LinearRegression(), X2_train, X2_test, y1_train, kfold)
    if(rms < temp):
        best_idx = i
        temp = rms
        best_rms = rms
    text = "Sử dụng "+ str(len(models[i])) + " đặc trưng " + str(col)
    print('{0: <5}'.format(i+1), '{0: <35}'.format(text), rms)
    



STT   Mô hình                             RMSE
1     Sử dụng 3 đặc trưng [0, 5, 6]       4.694664532385423
2     Sử dụng 1 đặc trưng [10]            4.532574940459634
3     Sử dụng 5 đặc trưng [0, 5, 6, 8, 11] 3.4382270440505067


In [648]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
my_best_model = model3
X2_train = X_train.iloc[:,my_best_model].to_numpy()
X2_test = X_test.iloc[:,my_best_model].to_numpy()

lr.fit(X2_train, y_train) 
y_predict = lr.predict(X2_test)

In [649]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
rms = rmse(y_predict, y_test)
rms

4.972446615817532

Công thức hồi quy

$$\text{Life expectancy} = 𝛽0 + 𝛽1x1 + 𝛽5x5 +𝛽6x6 +𝛽8x8 + 𝛽11𝑥11 + 𝜀$$